In [1]:
import pandas as pd
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go

cf.go_offline(connected= True)

In [2]:
df = pd.read_csv('okt_test.csv',encoding='cp949').iloc[1:,1:]

In [3]:
df

,Column2,Column3
1,"['[', '설빙', '](', '설빙', ')', '설빙', '디지털', '금액'...","['설빙', '설빙', '설빙', '디지털', '금액']"
2,"['[', '아웃백', '스테이크하우스', ']', '아웃백', '디지털', '5만...","['아웃백', '스테이크하우스', '아웃백', '디지털']"
3,"['(', '파리', '바게뜨', ')', '블레싱', '초코', '베리', '케이크']","['파리', '바게뜨', '블레싱', '초코', '베리', '케이크']"
4,"['[', '스타벅스', '](', '스타벅스', ')(', 'APP', '전용',...","['스타벅스', '스타벅스', '전용', '카드', '교환']"
5,"['(', '파리', '바게뜨', ')', '생', '딸기', '요거트', '케이크']","['파리', '바게뜨', '생', '딸기', '요거트', '케이크']"
...,...,...
10294,"['시원한', '스노우', '리플', '여름', '누비', '이불', '패드', '...","['스노우', '리플', '여름', '누비', '이불', '패드', '풀', '세트..."
10295,"['순면', '누빔', '고정', '밴딩', '여름', '침대', '패드', '/'...","['순면', '누빔', '고정', '밴딩', '여름', '침대', '패드', '인견..."
10296,"['[', '베드', '리움', '][', '금성', '침대', 'X', '베드',...","['베드', '리움', '금성', '침대', '베드', '리움', '베드', '리움..."
10297,"['네스트', '호텔', '인천', '(', '인천', '호텔', '/', '중구'...","['네스트', '호텔', '인천', '인천', '호텔', '중구', '인천', '국..."


In [4]:
from konlpy.tag import Okt
from konlpy.tag import Kkma

okt = Okt()
kkma = Kkma()

print('OKT 형태소 분석 :',okt.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 품사 태깅 :',okt.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 명사 추출 :',okt.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요")) 

OKT 형태소 분석 : ['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '가봐요']
OKT 품사 태깅 : [('열심히', 'Adverb'), ('코딩', 'Noun'), ('한', 'Josa'), ('당신', 'Noun'), (',', 'Punctuation'), ('연휴', 'Noun'), ('에는', 'Josa'), ('여행', 'Noun'), ('을', 'Josa'), ('가봐요', 'Verb')]
OKT 명사 추출 : ['코딩', '당신', '연휴', '여행']


# 한국어 단어 토큰화(우리의 title은 문장이 아니므로, 단어 토큰화만 진행하면 된다.) 를 실행하여 보았다. 일단은 명사만 추출하여 정리하여보자.

In [5]:
df1 = pd.read_csv('product.csv',header = None)

In [6]:
df_title =  df1.iloc[:,[3]]
df_title.columns = ['title']

In [7]:
df_title = df_title.drop_duplicates()

In [8]:
# def make_okt(row):
#     row['title'] = okt.nouns(row['title'])
#     return row
    
    
# df_title_spec = df_title.apply(make_okt,axis=1)

In [9]:
from tqdm import tqdm

word_count = {}


for text in tqdm(df_title['title']):
    for word in okt.nouns(text):
        word_count[word] = word_count.get(word,0) + 1
        
    

100%|████████████████████████████████████████████████████████████████████████████| 9968/9968 [00:09<00:00, 1001.28it/s]


In [10]:
import operator

sdict = sorted(word_count.items(), key=operator.itemgetter(1),reverse=True)


# 데이터 정제 ; 길이 1 제거

In [11]:
lst_word = []
for i,j in sdict:
    if len(i) <= 1:
        pass
    else:
        lst_word.append([i,j])

In [12]:
import konlpy
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
df_word =  pd.DataFrame(lst_word,columns = ['word', 'num'])
df_word

,word,num
0,세트,624
1,삼성,369
2,전자,367
3,여름,313
4,남성,310
...,...,...
9128,바란,1
9129,예은,1
9130,파르페,1
9131,슈페르,1


In [14]:
cv = CountVectorizer(vocabulary = df_word['word'],max_features= 3000,min_df= 3, max_df= 0.5)
print(cv)

CountVectorizer(max_df=0.5, max_features=3000, min_df=3,
                vocabulary=0        세트
1        삼성
2        전자
3        여름
4        남성
       ... 
9128     바란
9129     예은
9130    파르페
9131    슈페르
9132    거제시
Name: word, Length: 9133, dtype: object)


In [15]:
df_title

,title
0,(파리바게뜨) 실키롤케익
1,[설빙](설빙) 설빙 디지털 금액권 2만원권
2,[아웃백스테이크하우스]아웃백 디지털 5만원권
3,(파리바게뜨) 블레싱 초코베리 케이크
4,[스타벅스](스타벅스)(APP전용) e카드 3만원 교환권
...,...
10294,시원한스노우리플여름누비이불패드풀세트SSQK사이즈
10295,순면누빔 고정밴딩 여름침대패드/인견패드
10296,[베드리움][금성침대 X 베드리움] [베드리움] 슈페르 프리미엄 300T 매트리스(...
10297,네스트호텔 인천(인천 호텔/중구/인천국제공항(중구))


In [16]:
review_cv = cv.fit_transform(df_title['title'])

### DTM table 계산 계산 및 적용

In [17]:
print(cv.get_feature_names()[0:20])
print(review_cv.toarray())

['세트', '삼성', '전자', '여름', '남성', '블랙', '키즈', '여성', '박스', '신세계', '아이', '증정', '정품', '가방', '마스크', '무선', '화이트', '티셔츠', '프로', '아동']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



In [18]:
df_review_cv = pd.DataFrame(review_cv.toarray(),columns  =cv.get_feature_names() )

In [19]:
cv = CountVectorizer(max_features= 9000,min_df= 2, max_df= 0.7,)

In [20]:
test = cv.fit_transform(df_title['title'].tolist())

In [21]:
df_test = pd.DataFrame(test.toarray(), columns= cv.get_feature_names())

In [22]:
df_test

,00,000,0001,000원,001,00198,002,003,004,005,...,히어로,히키스,힌지보호,힐링,힐링그린,힐링팩토리,힐튼,힙분유,힙색,힙팩
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9965,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9966,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 데이터 정규화 & 정제

In [23]:
import re

def hasNumber(stringVal):
    if (re.search('\d', stringVal) == None):
        return False
    else:
        return True
    
    
def hasEnglish(stringVal):
    if (re.search('[A-Za-z]',stringVal) == None):
        return False
    else:
        return True

In [24]:
hasEnglish('a')

True

In [25]:
col_lst = []

for col in df_test.columns:
    if (hasNumber(col)) or (hasEnglish(col)):
        pass
    else:
        col_lst.append(col)
    

In [26]:
df_test_2 = df_test.loc[:,col_lst].copy()

In [27]:
word_value = df_test_2.iloc[:,1:].apply(sum,axis=0)

In [28]:
pd.DataFrame(word_value).sort_values(0,ascending=False)[0:1000].iplot('bar', title = '단어 별 발생 빈도 1000순위 까지')

In [29]:
lst_len = []
for i in df_test_2.columns:
    lst_len.append(len(i))

In [30]:
pd.Series(lst_len).value_counts().iplot('bar',title = '단어길이(x)에 따라 토큰화된 문자의 수')

In [31]:
df_test_2.insert(0,'Main_category',df1[0])

In [32]:
df_test_2

,Main_category,_편백나무,正品,防彈少年團,가게놀이,가격인하,가고,가그린,가나양말,가난한,...,히어로,히키스,힌지보호,힐링,힐링그린,힐링팩토리,힐튼,힙분유,힙색,힙팩
0,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9963,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9964,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9965,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9966,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
import numpy as np
random = np.random.random_integers(0,9967,size = int(9967* 0.2)).tolist()
df_test_3 = df_test_2.drop(random,axis=0).copy()
df_test_3

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning:

This function is deprecated. Please call randint(0, 9967 + 1) instead



,Main_category,_편백나무,正品,防彈少年團,가게놀이,가격인하,가고,가그린,가나양말,가난한,...,히어로,히키스,힌지보호,힐링,힐링그린,힐링팩토리,힐튼,힙분유,힙색,힙팩
1,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,e쿠폰/티켓,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9962,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9963,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9964,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9965,가구/침구,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
df_test_3.set_index('Main_category').apply(sum,axis=0)

_편백나무     3
正品        4
防彈少年團     2
가게놀이      2
가격인하      3
         ..
힐링팩토리     2
힐튼        3
힙분유       4
힙색       13
힙팩        2
Length: 5811, dtype: int64

In [35]:
name_lst  = [i for i in [df_test_3.set_index('Main_category').apply(sum,axis=0) > 0][0]]

In [45]:
name_lst

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 Tr

In [36]:
df_test_4 = df_test_3.set_index('Main_category').copy()

In [37]:
df_test_5 = df_test_4.loc[:,name_lst].copy()

In [39]:
# df_test_5.to_csv('TF.csv')

In [40]:
df1 = df1.drop_duplicates(3).copy()

In [41]:
df_title['Main_category'] = df1[0]

In [42]:
df_title_2 = df_title.set_index('Main_category').copy()

In [43]:
df_title_2

,title
Main_category,
e쿠폰/티켓,(파리바게뜨) 실키롤케익
e쿠폰/티켓,[설빙](설빙) 설빙 디지털 금액권 2만원권
e쿠폰/티켓,[아웃백스테이크하우스]아웃백 디지털 5만원권
e쿠폰/티켓,(파리바게뜨) 블레싱 초코베리 케이크
e쿠폰/티켓,[스타벅스](스타벅스)(APP전용) e카드 3만원 교환권
...,...
가구/침구,시원한스노우리플여름누비이불패드풀세트SSQK사이즈
가구/침구,순면누빔 고정밴딩 여름침대패드/인견패드
가구/침구,[베드리움][금성침대 X 베드리움] [베드리움] 슈페르 프리미엄 300T 매트리스(...


In [44]:
df_test_5.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_test_5.values, df_test_5.index, test_size=0.2)




### 

## TF 에서는 빈도 수 중 0인 것만 제거하여야 더 좋게 성능이 나온다.

In [ ]:
y_train

In [ ]:
from sklearn.naive_bayes import MultinomialNB #sklearn이 제공하는 MultinomialNB 를 사용
NB_clf = MultinomialNB() # 분류기 선언

NB_clf.fit(x_train, y_train) #train set을 이용하여 분류기(classifier)를 학습

print('Train set score: {:.3f}'.format(NB_clf.score(x_train, y_train))) #train set에 대한 예측정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(x_test, y_test))) #test set에 대한 예측정확도를 확인

In [ ]:
df_title_2

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_title_2,
                                                      df_title_2.index, test_size=0.2, shuffle=True, stratify=df_title_2.index
                                                     ,random_state=34)

In [ ]:
x_train['title'].values.tolist()

## TF-IDF를 적용시킬 때 각각 따로 해야한다는 것을 잊지 말자.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(max_features=2000,) 

tfidf_title_2 = tfidf.fit_transform(df_title_2['title'].tolist()).toarray()

pd.DataFrame(tfidf_title_2, columns = tfidf.get_feature_names_out())

In [ ]:
df_title_2.head()

In [ ]:
x_train

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=2000,) 
X_train_tfidf = tfidf.fit_transform(x_train['title'].values.tolist()) # train set을 변환
a = tfidf.get_feature_names_out()
X_test_tfidf = tfidf.transform(x_test['title'].values.tolist()) # test set을 변환
b = tfidf.get_feature_names_out()

In [ ]:
y_train

In [ ]:
import re

def hasNumber(stringVal):
    if (re.search('\d', stringVal) == None):
        return True
    else:
        return False
    
    
def hasEnglish(stringVal):
    if (re.search('[A-Za-z]',stringVal) == None):
        return True
    else:
        return False

In [ ]:
X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(),columns = tfidf.get_feature_names_out() )

In [ ]:
lst_bool = []
for i in X_train_tfidf_df.columns:
    if hasNumber(i) and hasEnglish(i):
        lst_bool.append(i)
        
    else:
        pass


In [ ]:
X_train_tfidf_df_2 = X_train_tfidf_df.loc[:,lst_bool].copy()

In [ ]:
X_train_tfidf_df_2.index = x_train.index

In [ ]:
X_test_tfidf_df = pd.DataFrame(X_test_tfidf.toarray(),columns = tfidf.get_feature_names_out() )

In [ ]:
lst_bool = []
for i in X_test_tfidf_df.columns:
    if hasNumber(i) and hasEnglish(i):
        lst_bool.append(i)
        
    else:
        pass

In [ ]:
X_test_tfidf_df_2 = X_test_tfidf_df.loc[:,lst_bool].copy()

In [ ]:
X_test_tfidf_df_2.index = x_test.index

In [ ]:
X_test_tfidf_df_2

In [ ]:
import pandas as pd
import numpy as np # 각 모델에서 내부적으로 관련 라이브러리 사용 가능

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

from sklearn.neighbors import KNeighborsClassifier             # 1. K-Nearest Neighbor(KNN)
from sklearn.linear_model import LogisticRegression            # 2. Logistic Regression
from sklearn.svm import SVC                                                # 3. SVC
from sklearn.tree import DecisionTreeClassifier                   # 4. Decision Tree
from sklearn.ensemble import RandomForestClassifier       # 5. Random Forest
from sklearn.ensemble import ExtraTreesClassifier             # 6. Extra Tree
from sklearn.ensemble import GradientBoostingClassifier  # 7. GBM
from sklearn.naive_bayes import GaussianNB                     # 8. GaussianNB
from xgboost import XGBClassifier                                     # 9. XGBoost
from lightgbm import LGBMClassifier                                 # 10. LightGBM
import warnings

warnings.filterwarnings('ignore')

In [ ]:
NB_clf.fit(X_train_tfidf_df_2, y_train) #tfidf train set을 이용하여 분류기(classifier)를 새로 학습
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf_df_2.values, y_train))) #train set에 대한 예측정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf_df_2.values, y_test))) #test set에 대한 예측정확도를 확인


In [ ]:
# from sklearn.tree import DecisionTreeClassifier


# tree3 = DecisionTreeClassifier(min_samples_split=50,random_state=7)
# #tree3 = DecisionTreeClassifier(random_state=7)
# #tree3 = DecisionTreeClassifier(max_depth=2,random_state=7)

# tree3.fit(X_train_tfidf_df_2.values, y_train)
# print('#Decision Tree train set score: {:.3f}'.format(tree3.score(X_train_tfidf_df_2.values, y_train)))
# print('#Decision Tree test set score: {:.3f}'.format(tree3.score(X_test_tfidf_df_2.values, y_test)))

## 여러 모델 적용 


In [ ]:


from sklearn.decomposition import PCA

### 말이 안된다. 로지스틱 회귀는 이진 분류기인데 어떻게 여러 개의 분류를......?

In [ ]:
from sklearn.linear_model import LogisticRegression 
LR_clf = LogisticRegression() #분류기 선언

LR_clf.fit(X_train_tfidf_df_2.values, y_train) # train data를 이용하여 분류기를 학습
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf_df_2.values, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf_df_2.values, y_test))) 

In [ ]:
# X_train_tfidf_df_2.to_csv('X_train_tfidf.csv')
# X_test_tfidf_df_2.to_csv('X_test_tfidf.csv')
# pd.DataFrame(y_train).to_csv('y_train.csv')
# pd.DataFrame(y_test).to_csv('y_test.csv')


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=750, random_state=7)
X_train_pca = pca.fit_transform(X_train_tfidf_df_2.values)
X_test_pca = pca.transform(X_test_tfidf_df_2.values)

print('Original tfidf matrix shape:', X_train_tfidf_df_2.shape)
print('PCA Converted matrix shape:', X_train_pca.shape)
print('Sum of explained variance ratio: {:.3f}'.format(pca.explained_variance_ratio_.sum()))

In [ ]:
X_test_pca

In [ ]:
from sklearn.decomposition import PCA



gd = GradientBoostingClassifier()

gd.fit(X_train_pca, y_train)


# xgb_clf = XGBClassifier()
# pca = PCA(n_components=1000, random_state=7)
# x.fit(X_train_pca, y_train)
# print('#Train set score: {:.3f}'.format(LR_clf.score()))
# print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_pca, y_test)))

In [ ]:
# import joblib 
# joblib.dump(gd, 'Gradient_boosting.pkl')

 

In [ ]:
print('#Train set score: {:.3f}'.format(gd.score(X_train_pca, y_train)))
print('#Test set score: {:.3f}'.format(gd.score(X_test_pca, y_test))) 



In [ ]:
from sklearn.preprocessing import MinMaxScaler #fixed import

#ValueError: Negative values in data passed to MultinomialNB (input X) 에러 방지
scaler = MinMaxScaler()
X_train_pca = scaler.fit_transform(X_train_pca)
X_test_pca = scaler.transform(X_test_pca)


n_estimators = [10, 50]
max_depth = [3, None] 
max_features = [0.1, 0.2, 0.5] # feature 수
min_samples_split = [2, 4, 6] # 노드를 분할하기 위한 최소 샘플 수
min_samples_leaf = [2, 4, 6] # 리프 노드가 되기 위해 필요한 최소 샘플 수

hyperparams = {
    'n_estimators': n_estimators, 
    'max_depth': max_depth, 
    'max_features': max_features,
    'min_samples_split': min_samples_split, 
    'min_samples_leaf': min_samples_leaf
}

RF=GridSearchCV(
    estimator = RandomForestClassifier(random_state=1), 
    param_grid = hyperparams, 
    verbose=True, 
    cv=5, 
    scoring = "accuracy", 
    n_jobs=-1
)


RF.fit(X_train_pca, y_train)


In [ ]:
# import joblib 
# joblib.dump(RF, 'RandomForest.pkl')

In [ ]:
print('#Train set score: {:.3f}'.format(RF.score(X_train_pca, y_train)))
print('#Test set score: {:.3f}'.format(RF.score(X_test_pca, y_test))) 


In [ ]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

## TF-IDF 계산

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer


transformer  = TfidfTransformer()

title_tfidf = transformer.fit_transform(review_cv)

In [ ]:
print('shape of tf-idf :' ,title_tfidf.shape)

In [ ]:
cv = CountVectorizer(vocabulary = df_word['word'],max_features= 3000,min_df= 3, max_df= 0.5)

In [ ]:
pd.DataFrame(title_tfidf.toarray())

In [ ]:
tfidf_df =pd.DataFrame(title_tfidf.toarray(),columns = cv.get_feature_names())

In [ ]:
tfidf_df.index = df1.drop_duplicates([3])[0]

In [ ]:
tfidf_df

In [ ]:
col_lst = []

for col in tfidf_df.columns:
    if (hasNumber(col)) or (hasEnglish(col)):
        pass
    else:
        col_lst.append(col)
    

In [ ]:
tfidf_df_2 = tfidf_df[col_lst].copy()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(tfidf_df_2,
                                                      tfidf_df_2.index, test_size=0.2, shuffle=True, stratify=tfidf_df_2.index, random_state=34)





In [ ]:
NB_clf.fit(x_train,y_train) #tfidf train set을 이용하여 분류기(classifier)를 새로 학습
print('Train set score: {:.3f}'.format(NB_clf.score(x_train, y_train))) #train set에 대한 예측정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(x_test, y_test))) #test set에 대한 예측정확도를 확인

### TF-IDF 먼저 나누고 계산하여 적용